In [1]:
import numpy as np
import os

In [9]:
local_model_path = '../wals_ml_engine/jobs/wals_ml_local_20180521_162349'
ROW_MODEL_FILE = 'model/row.npy'
COL_MODEL_FILE = 'model/col.npy'
USER_MODEL_FILE = 'model/user.npy'
ITEM_MODEL_FILE = 'model/item.npy'
RATINGS_FILE = 'model/ratings.npy'

In [225]:
user_factor = np.load(os.path.join(local_model_path, ROW_MODEL_FILE))
item_factor = np.load(os.path.join(local_model_path, COL_MODEL_FILE))
user_map = np.load(os.path.join(local_model_path, USER_MODEL_FILE))
item_map = np.load(os.path.join(local_model_path, ITEM_MODEL_FILE))
ratings = np.load(os.path.join(local_model_path, RATINGS_FILE))

In [226]:
user_map

array([      34348347166136,      124168996164483,      133046654287433,
       ..., 18313412406127452240, 18427461270502861788,
       18445572799162209077], dtype=uint64)

In [227]:
item_map

array([   709531,    709733,    709763, ..., 299987103, 299992213,
       299996869])

In [13]:
input_file = "../data/recommendation_events_new.csv"

In [15]:
new_events = []
mapped_events = []
new_users = []
new_items = []
with open(input_file) as f:
    next(f) #skip header
    for line in f:
      client_id, article_id, view_time = line.split(",")
      client_id = int(client_id)
      article_id = int(article_id)
      view_time = int(view_time)

      user_ix = np.searchsorted(user_map, client_id)
      item_ix = np.searchsorted(item_map, article_id)

      new_item_or_user = False
      if user_map[user_ix] != client_id:
        new_users.append(client_id)
        new_item_or_user = True
      if item_map[item_ix] != article_id:
        new_items.append(article_id)
        new_item_or_user = True

      if new_item_or_user:
        new_events.append((client_id, article_id, view_time))
      else:
        mapped_events.append((user_ix, item_ix, view_time))

In [313]:
all_users = np.concatenate((user_map, new_users))
sorted_user_ix = np.argsort(all_users)
all_user_factor = np.vstack((user_factor, np.zeros((len(new_users), user_factor.shape[1]))))
sorted_factors = all_user_factor[sorted_user_ix]
sorted_users = all_users[sorted_user_ix]

In [314]:
all_users

array([3.43483472e+13, 1.24168996e+14, 1.33046654e+14, ...,
       1.59911602e+18, 3.65656032e+18, 4.37725365e+18])

In [317]:
sorted_user_ix[-50:-1]

array([75982, 75983, 75984, 75985, 75986, 75987, 75988, 75989, 75990,
       77017, 75991, 75992, 75993, 75994, 75995, 77308, 75996, 75997,
       75998, 75999, 76000, 76001, 76002, 76003, 76004, 76005, 76006,
       76007, 76008, 76009, 76010, 76011, 76012, 76013, 76014, 76015,
       76016, 76017, 76018, 76019, 76020, 76021, 76022, 76023, 76024,
       76025, 76026, 76027, 76028])

In [319]:
user_map.shape

(76030,)

In [321]:
sorted_users[0]

34348347166136.0

In [322]:
user_map[0]

34348347166136

In [323]:
all_user_factor[0]

array([  7.49288368,  17.37949562,   4.49660206,  32.02349091,
       -17.68611908,  16.13059425,  -4.87479639, -14.35096741,
         9.40712261,  27.41927147,   5.56567717, -18.82164192,
        -1.48984659, -16.07185364,  22.93484497,  -7.18919563,
       -23.04388428, -24.3555088 ,   2.19006371,  -6.28728008,
         7.38154554,   9.39912128,  -8.44683933,  10.64288902,
       -20.13733864,  -3.8636899 ,  -1.27410769,   7.1074729 ,
        16.78817749,   8.38585949])

In [326]:
user_factor[0]

array([  7.4928837,  17.379496 ,   4.496602 ,  32.02349  , -17.68612  ,
        16.130594 ,  -4.8747964, -14.350967 ,   9.407123 ,  27.419271 ,
         5.565677 , -18.821642 ,  -1.4898466, -16.071854 ,  22.934845 ,
        -7.1891956, -23.043884 , -24.355509 ,   2.1900637,  -6.28728  ,
         7.3815455,   9.399121 ,  -8.446839 ,  10.642889 , -20.137339 ,
        -3.86369  ,  -1.2741077,   7.107473 ,  16.788177 ,   8.3858595],
      dtype=float32)

In [327]:
sorted_factors[0]

array([  7.49288368,  17.37949562,   4.49660206,  32.02349091,
       -17.68611908,  16.13059425,  -4.87479639, -14.35096741,
         9.40712261,  27.41927147,   5.56567717, -18.82164192,
        -1.48984659, -16.07185364,  22.93484497,  -7.18919563,
       -23.04388428, -24.3555088 ,   2.19006371,  -6.28728008,
         7.38154554,   9.39912128,  -8.44683933,  10.64288902,
       -20.13733864,  -3.8636899 ,  -1.27410769,   7.1074729 ,
        16.78817749,   8.38585949])